<a href="https://colab.research.google.com/github/j-chenn/COMP551_Project_1/blob/main/COMP551_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1: Acquire, pre-process and analyze the data
## Acquiring both datasets:
Dataset 1: [Search Trends](https://github.com/google-research/open-covid-19-data/blob/master/data/exports/search_trends_symptoms_dataset/README.mdhttps://)

Dataset 2: [COVID hospitalization cases](https://github.com/google-research/open-covid-19-data)

In [1]:
# Imports

import numpy as np
import pandas as pd
import warnings
import math
import statistics
from numpy import nanmedian, NaN
warnings.filterwarnings('ignore')


In [2]:
# the week of 08/24/2020 for the data collection
# Load into pandas dataframes
st_df = pd.read_csv('2020_US_weekly_symptoms_dataset.csv', low_memory=False)
hp_df = pd.read_csv('aggregated_cc_by.csv', low_memory=False)

## Preprocess the datasets

## Target Weeks range: 2020-03-09 to 2020-09-21


In [3]:
# Search trends dataset Part I

#TODO: Preprocessing, remove all symptoms that have all zero entries (clean COLUMN)
st_df = st_df.dropna(how='all', axis=1)

#Remove all rows not in the date of the week chosen (clean ROW)
st_df = st_df[st_df['date'] >= '2020-03-04']

nameList = list(st_df['sub_region_1']) #extract the region names from st_df database
nameList = list(dict.fromkeys(nameList))  #remove duplicates

In [4]:
# Hospitalization dataset Part I

#TODO: Preprocessing

#keep the hospitalization features and delete the rest  (clean COLUMN)
hp_df = hp_df[['open_covid_region_code','region_name','date', 'hospitalized_new']]

#select the regions that match the Search trends dataset (clean ROW)
hp_df= hp_df[hp_df.region_name.isin(nameList)]

#select the regions that have the valid date range (clean ROW)
hp_df = hp_df[(hp_df['date'] >= '2020-03-09') & (hp_df['date'] <= '2020-09-27')]

hp_df.reset_index(inplace = True) 
#print(hp_df.to_string())

In [5]:
# Hospitalization dataset Part II
# Here we want to group dates in the same week together as one date (the weekdate)
hp_df1 = hp_df
weekdate = '2020-03-09'

#This loop will update all the dates row by row
for i, n in hp_df1.iterrows():
    if (i%7 == 0):
        weekdate = n['date']  #first date of the week
    else:
        hp_df1.at[i,'date'] = weekdate

In [6]:
#sum up the hospitalized_vew for weekly
# we are only using this hp_df2 to rid regions that have insignificant hospitalized data, such as 0 for total hospitalization
def cleanRegions(df):
    hp_df2 = df
    f = dict.fromkeys(hp_df2.columns.difference(['region_name']), 'first')
    f['hospitalized_new'] = sum
    hp_df2 = hp_df2.groupby('region_name', as_index=False).agg(f)
    hp_df2 = hp_df2[hp_df2.hospitalized_new != 0]
    print(hp_df2.to_string())
    tmplist = list(hp_df2['region_name']) 
    tmplist = list(dict.fromkeys(tmplist))  
    return(tmplist)

#this nameList will be a new regions list that removes region with total of 0 hospitalization value for all its dates
nameList2 = cleanRegions(hp_df1)

#filter hp_df1 based on the nameList2 (clean ROW)
hp_df2= hp_df1[hp_df1.region_name.isin(nameList2)]

      region_name        date  hospitalized_new  index open_covid_region_code
3          Hawaii  2020-03-09             802.0  87097                  US-HI
4           Idaho  2020-03-09            1811.0  86678                  US-ID
5           Maine  2020-03-09             445.0  84953                  US-ME
6         Montana  2020-03-09             687.0  83701                  US-MT
7        Nebraska  2020-03-09            2279.0  83071                  US-NE
8   New Hampshire  2020-03-09             736.0  82860                  US-NH
9      New Mexico  2020-03-09            3435.0  82413                  US-NM
10   North Dakota  2020-03-09             815.0  83280                  US-ND
11   Rhode Island  2020-03-09            2725.0  80738                  US-RI
12   South Dakota  2020-03-09            1473.0  80311                  US-SD
15        Wyoming  2020-03-09             262.0  78166                  US-WY


In [7]:
# Hospitalization dataset Part III

# merge 7 week rows into 1 and sum up the hospitalized_new data
hp_df3 = hp_df2.groupby(['region_name','date'])['hospitalized_new'].apply(sum).reset_index()

In [8]:
# Search trends dataset Part II

# Drop unnecessary columns (open_covid region_code, country_region_code, country_region) (clean COLUMN)
st_df1 = st_df.drop(st_df.columns[[0, 1, 2]], axis=1)

# Filter st_df based on nameList2 (clean ROW)
st_df1= st_df1[st_df.sub_region_1.isin(nameList2)]
# print(st_df1)
# print(st_df1.shape)

#Filter columns so that every column have at least sp_num% of non-zero entries  (clean COLUMN)
sp_num = 0.24  #optimized ratio without tremendous loss of dataset
st_df2 = st_df1.dropna(thresh=sp_num*len(st_df), axis=1)


st_df2.reset_index(inplace = True) 
st_df2 = st_df2.drop(st_df2.columns[[0]], axis=1)

# print("after........." )
print(st_df2.head(50))
#print(st_df2.shape)

   sub_region_1 sub_region_1_code        date  symptom:Adrenal crisis  \
0        Hawaii             US-HI  2020-03-09                     NaN   
1        Hawaii             US-HI  2020-03-16                     NaN   
2        Hawaii             US-HI  2020-03-23                     NaN   
3        Hawaii             US-HI  2020-03-30                     NaN   
4        Hawaii             US-HI  2020-04-06                     NaN   
5        Hawaii             US-HI  2020-04-13                     NaN   
6        Hawaii             US-HI  2020-04-20                     NaN   
7        Hawaii             US-HI  2020-04-27                     NaN   
8        Hawaii             US-HI  2020-05-04                     NaN   
9        Hawaii             US-HI  2020-05-11                     NaN   
10       Hawaii             US-HI  2020-05-18                     NaN   
11       Hawaii             US-HI  2020-05-25                     NaN   
12       Hawaii             US-HI  2020-06-01      

In [9]:
#Normalization of the Search Trend Datasets:

#Find each symptom's median
#61 symptoms
medianList = []
for (columnName, columnData) in st_df2.iteritems(): 
    if (columnName != 'sub_region_1' and columnName != 'sub_region_1_code' and columnName != 'date'):
        m =  nanmedian(columnData.values) 
#         print("my median is", m)
        s = columnData.size
        i = 0
        for i in range(s):
            v = columnData.values[i]
            if (math.isnan(v)):
                columnData.values[i] = columnData.values[i]
            else:
#                 print("my val before:", columnData.values[i])
                columnData.values[i] = columnData.values[i]/m
#                 print("my val after:", columnData.values[i])
            i = i+1

# print(st_df2.shape)
# print(st_df2)

my median is 9.85
my val before: 15.44
my val after: 1.56751269035533
my val before: 15.85
my val after: 1.6091370558375635
my val before: 11.73
my val after: 1.1908629441624365
my val before: 14.68
my val after: 1.4903553299492387
my val before: 11.28
my val after: 1.1451776649746193
my val before: 14.33
my val after: 1.4548223350253808
my val before: 16.0
my val after: 1.6243654822335025
my val before: 14.02
my val after: 1.4233502538071066
my val before: 15.71
my val after: 1.5949238578680205
my val before: 17.69
my val after: 1.7959390862944165
my val before: 13.85
my val after: 1.4060913705583757
my val before: 9.55
my val after: 0.969543147208122
my val before: 14.18
my val after: 1.4395939086294416
my val before: 16.3
my val after: 1.6548223350253808
my val before: 14.23
my val after: 1.4446700507614214
my val before: 18.97
my val after: 1.9258883248730965
my val before: 11.03
my val after: 1.1197969543147208
my val before: 18.03
my val after: 1.8304568527918783
my val before: 1

my val after: 0.36874236874236876
my val before: 5.39
my val after: 0.4387464387464387
my val before: 5.18
my val after: 0.4216524216524216
my val before: 5.6
my val after: 0.4558404558404558
my val before: 14.52
my val after: 1.181929181929182
my val before: 17.2
my val after: 1.4000814000814
my val before: 17.58
my val after: 1.4310134310134308
my val before: 11.94
my val after: 0.9719169719169719
my val before: 17.4
my val after: 1.4163614163614162
my val before: 16.07
my val after: 1.3080993080993082
my val before: 20.42
my val after: 1.6621896621896624
my val before: 17.37
my val after: 1.413919413919414
my val before: 17.95
my val after: 1.461131461131461
my val before: 12.88
my val after: 1.0484330484330484
my val before: 15.4
my val after: 1.2535612535612535
my val before: 15.3
my val after: 1.2454212454212454
my val before: 16.57
my val after: 1.3487993487993488
my val before: 16.02
my val after: 1.304029304029304
my val before: 13.08
my val after: 1.0647130647130647
my val be

my val before: 48.52
my val after: 3.554578754578755
my val before: 39.8
my val after: 2.9157509157509156
my val before: 32.67
my val after: 2.3934065934065933
my val before: 27.73
my val after: 2.0315018315018314
my val before: 24.65
my val after: 1.8058608058608057
my val before: 23.21
my val after: 1.7003663003663003
my val before: 18.09
my val after: 1.3252747252747252
my val before: 22.26
my val after: 1.6307692307692307
my val before: 22.98
my val after: 1.6835164835164835
my val before: 15.51
my val after: 1.1362637362637362
my val before: 17.68
my val after: 1.295238095238095
my val before: 21.68
my val after: 1.5882783882783882
my val before: 18.67
my val after: 1.3677655677655678
my val before: 26.05
my val after: 1.9084249084249085
my val before: 22.08
my val after: 1.6175824175824174
my val before: 14.53
my val after: 1.0644688644688645
my val before: 14.86
my val after: 1.0886446886446886
my val before: 24.32
my val after: 1.7816849816849816
my val before: 16.92
my val aft

my val after: 1.1321428571428571
my val before: 15.25
my val after: 1.8154761904761905
my val before: 14.04
my val after: 1.6714285714285713
my val before: 13.66
my val after: 1.6261904761904762
my val before: 12.9
my val after: 1.5357142857142856
my val before: 16.75
my val after: 1.994047619047619
my val before: 13.48
my val after: 1.6047619047619048
my val before: 16.31
my val after: 1.9416666666666664
my val before: 5.78
my val after: 0.6880952380952381
my val before: 9.75
my val after: 1.1607142857142856
my val before: 8.64
my val after: 1.0285714285714287
my val before: 8.3
my val after: 0.9880952380952381
my val before: 5.71
my val after: 0.6797619047619047
my val before: 8.92
my val after: 1.061904761904762
my val before: 6.98
my val after: 0.830952380952381
my val before: 9.1
my val after: 1.0833333333333333
my val before: 6.69
my val after: 0.7964285714285715
my val before: 8.16
my val after: 0.9714285714285714
my val before: 7.54
my val after: 0.8976190476190475
my val befor

my val before: 10.0
my val after: 1.1527377521613833
my val before: 9.67
my val after: 1.1146974063400574
my val before: 10.15
my val after: 1.170028818443804
my val before: 9.18
my val after: 1.0582132564841498
my val before: 3.99
my val after: 0.4599423631123919
my val before: 3.41
my val after: 0.3930835734870317
my val before: 3.28
my val after: 0.3780979827089337
my val before: 5.65
my val after: 0.6512968299711815
my val before: 4.64
my val after: 0.5348703170028818
my val before: 4.07
my val after: 0.469164265129683
my val before: 4.38
my val after: 0.5048991354466859
my val before: 4.7
my val after: 0.5417867435158501
my val before: 6.1
my val after: 0.7031700288184437
my val before: 3.47
my val after: 0.39999999999999997
my val before: 3.95
my val after: 0.4553314121037464
my val before: 4.06
my val after: 0.46801152737752155
my val before: 3.58
my val after: 0.4126801152737752
my val before: 3.58
my val after: 0.4126801152737752
my val before: 3.86
my val after: 0.44495677233

...

my val after: 1.7017969888295288
my val before: 11.21
my val after: 1.088878096163186
my val before: 16.05
my val after: 1.5590092277804761
my val before: 14.68
my val after: 1.4259349198640117
my val before: 11.73
my val after: 1.139388052452647
my val before: 15.85
my val after: 1.5395823215152986
my val before: 17.23
my val after: 1.673627974745022
my val before: 15.73
my val after: 1.5279261777561923
my val before: 21.61
my val after: 2.099077221952404
my val before: 18.2
my val after: 1.7678484701311317
my val before: 17.79
my val after: 1.7280233122875182
my val before: 15.8
my val after: 1.5347255949490044
my val before: 12.05
my val after: 1.1704711024769305
my val before: 13.02
my val after: 1.2646915978630402
my val before: 13.37
my val after: 1.2986886838271006
my val before: 14.27
my val after: 1.3861097620203982
my val before: 17.37
my val after: 1.6872268091306462
my val before: 16.9
my val after: 1.6415735794074793
my val before: 13.99
my val after: 1.35891209324915
my v

my val before: 34.89
my val after: 2.951776649746193
my val before: 31.45
my val after: 2.6607445008460235
my val before: 35.62
my val after: 3.0135363790186123
my val before: 37.09
my val after: 3.1379018612521152
my val before: 41.01
my val after: 3.4695431472081215
my val before: 33.0
my val after: 2.7918781725888326
my val before: 34.47
my val after: 2.9162436548223347
my val before: 29.69
my val after: 2.511844331641286
my val before: 36.2
my val after: 3.0626057529610833
my val before: 20.5
my val after: 1.7343485617597292
my val before: 39.92
my val after: 3.3773265651438242
my val before: 28.05
my val after: 2.3730964467005076
my val before: 33.13
my val after: 2.8028764805414554
my val before: 36.26
my val after: 3.067681895093062
my val before: 26.34
my val after: 2.228426395939086
my val before: 34.47
my val after: 2.9162436548223347
my val before: 29.81
my val after: 2.521996615905245
my val before: 30.79
my val after: 2.604906937394247
my val before: 25.17
my val after: 2.

my val after: 0.739656912209889
my val before: 9.87
my val after: 0.9959636730575175
my val before: 8.88
my val after: 0.8960645812310798
my val before: 6.84
my val after: 0.6902119071644803
my val before: 7.93
my val after: 0.8002018163471241
my val before: 7.73
my val after: 0.7800201816347124
my val before: 10.3
my val after: 1.039354187689203
my val before: 10.09
my val after: 1.0181634712411705
my val before: 8.26
my val after: 0.8335015136226034
my val before: 9.79
my val after: 0.9878910191725528
my val before: 9.22
my val after: 0.9303733602421796
my val before: 11.06
my val after: 1.1160443995963674
my val before: 10.44
my val after: 1.0534813319878908
my val before: 9.91
my val after: 1.0
my val before: 10.59
my val after: 1.0686175580221997
my val before: 10.19
my val after: 1.0282542885973762
my val before: 8.17
my val after: 0.8244197780020182
my val before: 8.96
my val after: 0.9041372351160445
my val before: 8.65
my val after: 0.8728557013118062
my val before: 6.64
my va

my val before: 10.55
my val after: 1.2084765177548682
my val before: 9.41
my val after: 1.0778923253150057
my val before: 10.89
my val after: 1.2474226804123711
my val before: 11.53
my val after: 1.3207331042382588
my val before: 11.64
my val after: 1.3333333333333333
my val before: 14.83
my val after: 1.6987399770904925
my val before: 11.95
my val after: 1.3688430698739975
my val before: 12.91
my val after: 1.4788087056128292
my val before: 12.68
my val after: 1.4524627720504009
my val before: 9.92
my val after: 1.13631156930126
my val before: 12.52
my val after: 1.4341351660939288
my val before: 8.73
my val after: 1.0
my val before: 9.69
my val after: 1.1099656357388314
my val before: 15.19
my val after: 1.7399770904925542
my val before: 11.57
my val after: 1.3253150057273768
my val before: 9.59
my val after: 1.0985108820160365
my val before: 10.61
my val after: 1.2153493699885451
my val before: 8.5
my val after: 0.9736540664375716
my val before: 8.4
my val after: 0.9621993127147767


my val before: 32.2
my val after: 2.5254901960784317
my val before: 35.33
my val after: 2.7709803921568628
my val before: 22.17
my val after: 1.7388235294117649
my val before: 18.97
my val after: 1.487843137254902
my val before: 12.79
my val after: 1.0031372549019608
my val before: 9.93
my val after: 0.7788235294117647
my val before: 8.29
my val after: 0.6501960784313725
my val before: 5.11
my val after: 0.4007843137254902
my val before: 5.04
my val after: 0.39529411764705885
my val before: 6.1
my val after: 0.4784313725490196
my val before: 5.12
my val after: 0.4015686274509804
my val before: 5.58
my val after: 0.43764705882352944
my val before: 5.2
my val after: 0.40784313725490196
my val before: 6.16
my val after: 0.4831372549019608
my val before: 6.28
my val after: 0.49254901960784314
my val before: 4.85
my val after: 0.3803921568627451
my val before: 19.17
my val after: 1.503529411764706
my val before: 18.58
my val after: 1.457254901960784
my val before: 13.76
my val after: 1.0792

## Merging the datasets 

In [10]:
hpData = hp_df3["hospitalized_new"]
hpData = pd.Series(hpData)

st_df2['hospitalized_new'] = hpData.values # Merging the data_set
print(st_df2)

    sub_region_1 sub_region_1_code        date  symptom:Adrenal crisis  \
0         Hawaii             US-HI  2020-03-09                     NaN   
1         Hawaii             US-HI  2020-03-16                     NaN   
2         Hawaii             US-HI  2020-03-23                     NaN   
3         Hawaii             US-HI  2020-03-30                     NaN   
4         Hawaii             US-HI  2020-04-06                     NaN   
..           ...               ...         ...                     ...   
314      Wyoming             US-WY  2020-08-24                0.448731   
315      Wyoming             US-WY  2020-08-31                0.510660   
316      Wyoming             US-WY  2020-09-07                0.341117   
317      Wyoming             US-WY  2020-09-14                0.421320   
318      Wyoming             US-WY  2020-09-21                0.572589   

     symptom:Allergic conjunctivitis  symptom:Angular cheilitis  \
0                                NaN        

In [11]:
# Convert merged dataset into a numpy array
myarray = pd.DataFrame(st_df2).to_numpy()

## End of Task 1